In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,48381
2,Huelva,Confirmados PDIA 14 días,1274
3,Huelva,Tasa PDIA 14 días,"248,2608102578093"
4,Huelva,Confirmados PDIA 7 días,624
5,Huelva,Tasa PDIA 7 dias,"121,59713155484538"
6,Huelva,Total Confirmados,48588
7,Huelva,Curados,45935
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  48381.0


/tmp/ipykernel_3671/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13466.0


/tmp/ipykernel_3671/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3671/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3671/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3671/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 719 personas en los últimos 7 días 

Un positivo PCR cada 333 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,48381.0,624.0,1274.0,513170.0,121.597132,248.260810,380.0
Huelva-Costa,28563.0,331.0,706.0,289548.0,114.316106,243.828312,204.0
Huelva (capital),13466.0,200.0,432.0,143837.0,139.046282,300.339968,121.0
Condado-Campiña,15102.0,208.0,425.0,156231.0,133.136189,272.033079,118.0
Sierra de Huelva-Andévalo Central,4286.0,83.0,141.0,67391.0,123.161847,209.226751,56.0
Moguer,2166.0,40.0,79.0,21867.0,182.924041,361.274981,30.0
Valverde del Camino,925.0,35.0,57.0,12750.0,274.509804,447.058824,24.0
Almonte,2411.0,34.0,46.0,24507.0,138.735871,187.701473,24.0
Aljaraque,1738.0,34.0,68.0,21474.0,158.331005,316.662010,24.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),18.0,1.0,8.0,243.0,411.522634,3292.181070,1.0
Puerto Moral,10.0,5.0,5.0,278.0,1798.561151,1798.561151,2.0
Paterna del Campo,280.0,17.0,55.0,3457.0,491.755858,1590.974834,5.0
Corteconcepción,50.0,3.0,4.0,536.0,559.701493,746.268657,1.0
Palos de la Frontera,1210.0,28.0,58.0,11742.0,238.460228,493.953330,15.0
Bollullos Par del Condado,1407.0,29.0,66.0,14387.0,201.570863,458.747480,15.0
Valverde del Camino,925.0,35.0,57.0,12750.0,274.509804,447.058824,24.0
Cumbres Mayores,142.0,5.0,7.0,1749.0,285.877644,400.228702,3.0
Higuera de la Sierra,75.0,2.0,5.0,1291.0,154.918668,387.296669,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Granada de Río-Tinto (La),18.0,1.0,8.0,243.0,411.522634,3292.181070,1.0,0.125000
Escacena del Campo,240.0,1.0,6.0,2287.0,43.725404,262.352427,0.0,0.166667
Ayamonte,2133.0,4.0,17.0,21104.0,18.953753,80.553450,3.0,0.235294
Rociana del Condado,697.0,5.0,21.0,7939.0,62.980224,264.516942,4.0,0.238095
Alosno,252.0,3.0,12.0,3933.0,76.277651,305.110603,3.0,0.250000
San Juan del Puerto,781.0,3.0,10.0,9411.0,31.877590,106.258634,3.0,0.300000
Paterna del Campo,280.0,17.0,55.0,3457.0,491.755858,1590.974834,5.0,0.309091
Villarrasa,236.0,1.0,3.0,2211.0,45.228403,135.685210,0.0,0.333333
Zalamea la Real,130.0,2.0,6.0,3054.0,65.487885,196.463654,1.0,0.333333
